In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [2]:
csv = pd.read_csv('updated2.csv', encoding='latin-1')
csv.head()

,date,AverageTemperatureF,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude,year
0,1743-11-01,42.9224,6.068,1.737,ÃÃÃÃ¤rhus,Denmark,0.80N,124.55E,1869
1,1744-04-01,42.4184,5.788,3.624,ÃÃÃÃ¤rhus,Denmark,0.80N,124.55E,1878
2,1744-05-01,51.1592,10.644,1.283,ÃÃÃÃ¤rhus,Denmark,0.80N,124.55E,1878
3,1744-06-01,57.2918,14.051,1.347,ÃÃÃÃ¤rhus,Denmark,0.80N,124.55E,1878
4,1744-07-01,60.9476,16.082,1.396,ÃÃÃÃ¤rhus,Denmark,0.80N,124.55E,1878


In [3]:
csv = csv.dropna()

In [4]:
df = pd.DataFrame(csv)
 
df['City'].nunique()

435

In [5]:
df['Country'].nunique()

88

In [6]:
df['date'].nunique()

3002

In [7]:
df['Longitude'] = df['Longitude'].str.replace('E', '')
df['Longitude'] = df['Longitude'].str.replace('W', '')
df['Latitude'] = df['Latitude'].str.replace('N', '')
df.to_csv('final.csv')

In [8]:
df.columns=['date', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
       'Country', 'Latitude', 'Longitude', 'year', 'AverageTemperatureF']

In [9]:
df.groupby(['year'])


In [10]:
# Assign X (data) and y (target)
X = df[['date', 'AverageTemperature', 'AverageTemperatureUncertainty', 'City',
        'Country', 'Latitude', 'Longitude', 'year']]
y = df["AverageTemperatureF"].values.reshape(-1, 1)
print(X.shape, y.shape)

(1048575, 8) (1048575, 1)


In [11]:
# Split the data into training and testing
from sklearn.model_selection import train_test_split

X = pd.get_dummies(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()

,AverageTemperature,AverageTemperatureUncertainty,City,date_1/1/00,date_1/1/01,date_1/1/02,date_1/1/03,date_1/1/04,date_1/1/05,date_1/1/06,...,year_91.75,year_92.13,year_93.50,year_94.15,year_95.25,year_96.70,year_97.12,year_97.85,year_98.84,year_99.91
240257,73.6574,23.143,0.371,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
323839,68.8280,20.460,0.620,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
233260,76.0910,24.495,0.529,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
879942,77.8532,25.474,1.505,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
90535,74.3108,23.506,0.969,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
from sklearn.preprocessing import StandardScaler

# # Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)


/Users/abigailvasquez/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
# # Transform the training and testing data using the X_scaler and y_scaler models
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

/Users/abigailvasquez/anaconda3/envs/PythonData/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [ ]:
# Create a LinearRegression model and fit it to the scaled training data
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)

In [ ]:
# Make predictions using the X_test_scaled data
# Plot y_test_scaled vs y_test_scaled
# Scatter plot y_test_scaled vs predictions

predictions = model.predict(X_test_scaled)
# scale predicted y values back to original domain
predictions = y_scaler.inverse_transform(predictions)
plt.scatter(model.predict(X_train_scaled), model.predict(X_train_scaled) - y_train_scaled, c="blue", label="Training Data")
plt.scatter(model.predict(X_test_scaled), model.predict(X_test_scaled) - y_test_scaled, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_test_scaled.min(), xmax=y_test_scaled.max())
plt.title("Residual Plot")
plt.show()
